<a href="https://colab.research.google.com/github/Jin0331/TA/blob/master/Big_data_Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **빅데이터개론 Lab 9 - 다중회귀분석(Multiple Linear Regression)**

- - -

참고자료 : https://www.notion.so/1e83a70a95474931beb46e23b8289b70




### **<다중선형회귀분석>**

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F99A0813C5E1D430625C52A" width="500" height="300">

* **Multiple linear regression (다중회귀분석)**

  : 다중회귀분석은 설명변수(독립변수)가 2개 이상인 회귀분석

  : 다중회귀분석에서 설명변수 개수가 많을 경우, 적절한 회귀모형 선택이 필요함

  : 회귀모형에 포함되는 설명변수의 선정 기준
      ㄱ) 반응변수(종속변수)와 높은 상관관계
      ㄴ) 선택된 설명변수들은 서로 낮은 상관관계를 보임(다중공선성 문제)
      ㄷ) 설명변수의 개수는 적을수록 유리

      * 다중공선성 관련 링크 : https://learnx.tistory.com/entry/%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1Multicollinearity%EC%9D%B4%EB%9E%80

<br>

* **Feature selection (변수 선택법)**

  1) All possible regressions

  - 변수들의 가능한 모든 조합들로부터 최적의 모형을 찾아냄

  - 유의한 변수가 누락되지 않는 안전한 방법

  - 변수가 많을수록 탐색 시간이 급증함

  2) Forward stepwise selection (Forward selection)

  - 기여도가 높은 유의한 변수부터 하나씩 추가하는 기법

  - 빠른 계산이 장점

  - 이미 선택된 변수는 다시 제거되지 않음

  3) Backward stepwise selection (Backward elimination)

  - 모든 변수를 포함한 상태에서 불필요한 변수를 제거해나가는 방법

  - 중요한 변수가 제외될 가능성이 매우 적음

  - 이미 제외된 변수는 다시 선택되지 않음
  
  <br>

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile23.uf.tistory.com%2Fimage%2F999339425E1D4A5626C373" width="600" height="500">

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F993971425E1D4A572FB764" width="600" height="500">


In [ ]:
install.packages(c("tidyverse", "data.table", "devtools", "scatterplot3d", "plotly","PerformanceAnalytics"))
library(tidyverse)
library(data.table)
library(devtools)
options(repr.plot.width=13, repr.plot.height=13)

- - -

#### **A. 예제 1**

https://github.com/kassambara/datarium

In [ ]:
install_github("kassambara/datarium")
library(datarium)

In [ ]:
data("marketing", package = "datarium")
marketing %>% as_tibble() %>% show()

In [ ]:
str(marketing)

* boxplot

In [ ]:
# boxplot을 이용한 outlier 확인 및 제거(?)
boxplot(marketing)

In [ ]:
str(boxplot(marketing, plot = F))

* scatter plot

In [ ]:
library(PerformanceAnalytics)
chart.Correlation(marketing, histogram=TRUE, pch=19)

In [ ]:
library(scatterplot3d)
scatterplot3d(marketing$youtube, marketing$facebook, marketing$sales, angle = 40)

In [ ]:
# plotly를 이용한 3d Sactterplot(https://plotly.com/r/3d-scatter-plots/)
# https://m.blog.naver.com/PostView.nhn?blogId=pmw9440&logNo=221590897819&proxyReferer=https:%2F%2Fwww.google.com%2F
# ftp://210.115.229.94/plotly_3d_scatter.html
library(plotly)
fig <- plot_ly(marketing, x = ~youtube, y = ~facebook, z = ~sales, color = ~sales)
fig <- fig %>% add_markers()
fig <- fig %>% layout(scene = list(xaxis = list(title = 'YOUTUBE'),
                                   yaxis = list(title = 'FACEBOOK'),
                                   zaxis = list(title = 'SALES')))
                        
htmlwidgets::saveWidget(fig, file = "3d_scatter.html")

* **회귀식 추정**
``sales = b0 + b1 * youtube + b2 * facebook + b3 * newspaper ``


In [ ]:
model <- lm(sales ~ youtube + facebook + newspaper, data = marketing)
model

* 회귀식 도출
* ```
b0 = 3.52667, b1 = 0.045765, b2 = 0.188530, b3 = -0.001037 으로부터 
sales = 3.52667 + 0.045765 * youtube + 0.188530 * facebook + -0.001037 * newspaper
```


* youtube, facebook, newspaper이 각각 1 증가할 때, sales가 0.045765, 0.188530, -0.001037이 증가

In [ ]:
summary(model)

<결과의 해석>
```
** m1 모델 **
1. 제일 밑에 F-statistic의 p-value 값은 2.2e-16 로 0.05보다 작기에 이 회귀식은 회귀분석 모델 전체에 대해 통계적으로 의미가 있다고 볼 수 있다.

2. 중간의 Coefficients:에는 y절편 값(Intercept) 및 변수들의 p-value 값이 나와있다. 
newspaper 변수의 경우 0.86으로 0.05보다 크기에 sales를 설명하는데 유의하지 않다.

3. 밑 부분의 Adjusted R-squared 값은 0.8956로, 89%만큼의 설명력을 가진다고 판단할 수 있다.
```

**<유의하지 않은 변수 제거>**

  * 위 다중회귀분석 결과에서 age와 ptl 변수가 유의하지 않은 것으로 확인됨

  * 아래의 3가지 방법 중 하나로, 유의하지 않은 변수를 제거할 수 있음

```
1) 유의하지 않은 변수를 누락하고 회귀모형을 새롭게 정의
model_2 <- lm(sales ~ youtube + facebook, data = marketing)

2) update 함수를 사용하여, 기존 회귀모형에서 유의하지 않은 변수 제거
model_2 <- update(model, .~. -newspaper)

3) step 함수를 사용하여, 기존 회귀모형에서 유의하지 않은 변수를 제거해나감
model_2 <- step(model, direction = "backward")
```

In [ ]:
# 1 번 방법
lm(sales ~ youtube + facebook, data = marketing)

In [ ]:
# 2 번 방법(update 사용)
update(model, .~. -newspaper)

In [ ]:
# 3 번 방법(step)
step(model, direction = "backward")

In [ ]:
# 변수의 상대적 중요도 파악
# https://rpubs.com/cardiomoon/152770
factor2number=function(df){
    select=sapply(df,function(x) {is.factor(x)})
    if(sum(select)==1) df[[which(select)]]<-as.numeric(df[[which(select)]])
    else if(sum(select)>1) df[,select]<-lapply(df[,select],as.numeric)
    df
}

chr2factor=function(df){
    select=sapply(df,function(x) {is.character(x)})
    df[,select]<-lapply(df[,select],factor)
    df
}

relweights <- function(fit,plot=FALSE,...){
    df<-fit$model
    df1<-chr2factor(df)
    df2<-factor2number(df1)
    
    R <- cor(df2)
    nvar <- ncol(R)
    rxx <- R[2:nvar, 2:nvar]
    rxy <- R[2:nvar, 1]
    svd <- eigen(rxx)
    evec <- svd$vectors
    ev <- svd$values
    delta <- diag(sqrt(ev))
    lambda <- evec %*% delta %*% t(evec)
    lambdasq <- lambda ^ 2
    beta <- solve(lambda) %*% rxy
    rsquare <- colSums(beta ^ 2)
    rawwgt <- lambdasq %*% beta ^ 2
    import <- (rawwgt / rsquare) * 100
    import <- as.data.frame(import)
    row.names(import) <- names(fit$model[2:nvar])
    names(import) <- "Weights"
    import <- import[order(import),1, drop=FALSE]
    if(plot) {
        dotchart(import$Weights, labels=row.names(import),
                 xlab="% of R-Square", pch=19,
                 main="Relative Importance of Predictor Variables",
                 sub=paste("Total R-Square=", round(rsquare, digits=3)),
                 ...)
    }  
    attr(import,"R-square")<-round(rsquare, digits=3)
    return(import)
}

plotRelWeights=function(fit){
       data<-relweights(fit)
       data$Predictors<-rownames(data)
       p<-ggplot(data=data,aes(x=reorder(Predictors,Weights),y=Weights,fill=Predictors))+ 
           geom_bar(stat="identity",width=0.5)+
           ggtitle("Relative Importance of Predictor Variables")+
           ylab(paste0("% of R-square \n(Total R-Square=",attr(data,"R-square"),")"))+
           geom_text(aes(y=Weights-0.1,label=paste(round(Weights,1),"%")),hjust=1)+
           guides(fill=FALSE)+
           coord_flip()
       p
   }



In [ ]:
plotRelWeights(model)

In [ ]:
model_2 <- step(model, direction = "backward")
plotRelWeights(model_2)

In [ ]:
# 3d 산점도
library(scatterplot3d)

s <- scatterplot3d(marketing$youtube, marketing$facebook, marketing$sales, pch = 20, type = 'h', angle = 55, color = "tomato")
s$plane3d(model_2)

* **예측**

In [ ]:
# test 데이터 생성
set.seed(51)
youtube_r <- sample(min(marketing$youtube):max(marketing$youtube), 60, replace = F)
facebook_r <- sample(min(marketing$facebook):max(marketing$facebook), 60, replace = F)

new_data <- tibble(youtube = youtube_r, facebook = facebook_r)
new_data

In [ ]:
# predict
sales_r <- predict(model_2, new_data)
new_data <- new_data %>% bind_cols(tibble(sales_predict = sales_r))
new_data %>% show()

In [ ]:
# Add supplementary points
s <- scatterplot3d(marketing$youtube, marketing$facebook, marketing$sales, pch = 20, type = 'h', angle = 50, color = "grey")
s$plane3d(model_2)
s$points3d(new_data$youtube, new_data$facebook, new_data$sales_predict, col = "red", type = "h", pch = 16, cex = 1.5)

In [ ]:
s <- scatterplot3d(marketing$youtube, marketing$facebook, marketing$sales, pch = 20, type = 'h', angle = 25, color = "grey")
s$plane3d(model_2)
s$points3d(new_data$youtube, new_data$facebook, new_data$sales_predict, col = "red", type = "h", pch = 16, cex = 1.5)

- - -

#### **B. 예제 2**

https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=dateDown&view=table

In [ ]:
library(data.table)
library(tidyverse)

Sys.setlocale('LC_ALL','C') # 이부분 무시해도 됨.
DF <- fread("ftp://210.115.229.94/2020_datascience/BikeSeoul/SeoulBikeData.csv", sep = ",", stringsAsFactors = F) %>%
 as_tibble()

In [ ]:
DF %>% show()

In [ ]:
DF %>% str()

* 열(column) 이름 변경

In [ ]:
#https://datascience.stackexchange.com/questions/8922/removing-strings-after-a-certain-character-in-a-given-text
#https://stackoverflow.com/questions/5992082/how-to-remove-all-whitespace-from-a-string

colnames(DF) <- colnames(DF) %>% lapply(X = ., FUN = function(value){
  stringr::str_remove(string = value, pattern = "\\(.*") %>%
  stringr::str_replace_all(string = ., pattern = "[[:space:]]", replace = "_")
}) %>% unlist()

DF %>% str()

In [ ]:
DF %>% summary()

* 문자형 데이터를 범주형 데이터로 변환

In [ ]:
DF %>% boxplot()

In [ ]:
DF <- DF %>% mutate_if(is.character, factor)
str(DF)

DF %>% boxplot(las=2)
DF %>% boxplot(plot = F) %>% str()

In [ ]:
# GGally의 ggpairs를 이용한 산점도 및 상관관계 확인
install.packages("GGally")
DF %>% select_at(-1) %>% GGally::ggpairs()

* train / test 분리

In [ ]:
install.packages(c("caTools", "caret"))

In [ ]:
library(caret)
library(caTools)

In [ ]:
# caret을 이용한 train / test data 분리
?createDataPartition
trainIndex <- caret::createDataPartition(1:nrow(DF), p = 0.80, list = F)
head(trainIndex)

In [ ]:
DF_train <- DF[trainIndex, ] # 0.8
DF_test <- DF[-trainIndex, ] # 0.2

DF %>% nrow()
DF_train %>% show()
DF_test %>% show()

In [ ]:
# caTools 이용한 train / test data 분리
?sample.split
trainIndex <- caTools::sample.split(1:nrow(DF), SplitRatio = 0.80)
trainIndex

In [ ]:
DF_train <- filter(DF, trainIndex == T)
DF_test <- filter(DF, trainIndex == F)

DF_train %>% show()
DF_test %>% show()

* **회귀식 추정**

In [ ]:
str(DF_train)

In [ ]:
summary(lm(Rented_Bike_Count  ~ Seasons + Holiday, data = DF_train))
model.matrix(lm(Rented_Bike_Count  ~ Seasons + Holiday, data = DF_train)) %>% as_tibble() %>% show()

* Date(날짜)를 제외한 모든 설명변수를 이용한 회귀분석

In [ ]:
# Hour --> 오전(0 ~ 12), 오후(13 ~24)으로 그룹화(0 == 오전, 1 == 오후)
DF_train <- DF_train %>% mutate(Hour = ifelse(Hour <= 12, 0, 1)) %>%
 mutate(Hour = factor(Hour))
DF_test <- DF_test %>% mutate(Hour = ifelse(Hour <= 12, 0, 1)) %>%
 mutate(Hour = factor(Hour))

In [ ]:
DF_train %>% select(Hour) %>% show()

In [ ]:
model <- lm(Rented_Bike_Count ~ . -Date , data = DF_train)
summary(model)

In [ ]:
plotRelWeights(model)

* step을 이용한 방법

In [ ]:
model_2 <- step(model, direction = "backward")
summary(model_2)

In [ ]:
plotRelWeights(model_2)

* Wind_speed_ 제거

In [ ]:
model_3 <- update(model_2, .~. -Wind_speed_)
summary(model_3)
plotRelWeights(model_3)

* Observed/fitted values and residuals

In [ ]:
install.packages("moderndive")

In [ ]:
set.seed(30)
moderndive::get_regression_points(model_3) %>%
 sample_n(size = 40) %>% select(ID, Rented_Bike_Count, Rented_Bike_Count_hat, residual)

* 사용되는 설명변수의 수가 많을 수록 결정계수는 상승한다..!

In [ ]:
summary(lm(Rented_Bike_Count ~ Dew_point_temperature + Hour + Humidity, data = DF_train))

* test를 이용한 예측

In [ ]:
predict(model_3, DF_test)

* Observed/fitted values and residuals

In [ ]:
set.seed(50)
DF_test %>% select(Rented_Bike_Count) %>% 
 bind_cols(., tibble(Rented_Bike_Count_test_hat = predict(model_3, DF_test))) %>%
 mutate(residual = Rented_Bike_Count - Rented_Bike_Count_test_hat) %>%
 sample_n(size = 50)

- - -

## **C. 실습 과제 <9주차 과제 - 100점>**

### **(1) 각각의 문제 바로 아래에 답을 작성 후, Big_data_Lab9.ipynb 파일을 한림SmartLEAD에 제출할 것**
### **(2) 제출 데드라인 : 2020.10.26(월요일) ~ 2020.11.1(일요일) 23:59 까지**
### **(3) 부정행위 적발 시, 원본(보여준 사람)과 복사본(베낀 사람) 모두 0점 처리**
### **(4) !!해당 문제의 코드와 출력 결과를 반드시 기록할 것(둘 중 하나라도 없으면 해당 문제 0점처리)!!**


- - -


### **1. 다음의 [링크(클릭)](https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table)에서 Default Task가 ``	Regression ``인 데이터를 적절히 선택하여, ``단순회귀분석(Simple Linear Regression)``과 ``다중회귀분석(Multiple Linear Regression)``을 실습자료의 ``예제1과 예제2 유사한 방식``으로 진행하시오.**

``* 단순회귀분석 50점, 다중회귀분석 50점``

<br>

#### **다음의 조건을 만족해야 함**

```
## 조건
1. 선택한 데이터의 Attribute(속성 또는 열)에 대한 설명
2. scatter plot(산점도), boxplot 등 시각화하여 각 Attribute의 값에 대한 설명
3. 적절한 반응변수와 설명변수의 섫정
4. Train / Test set 분리 후 회귀식 추정
5. 회귀분석 결과 해석
6. Feature selection(변수 선택)
7. Test set을 이용한 Predict(예측)
```



### **1-1. 단순회귀분석 (50점)**

* 선택한 데이터의 Attribute(속성 또는 열)에 대한 설명

```
(여기에 작성!!!)
(여기에 작성!!!)
(여기에 작성!!!)
(여기에 작성!!!)
(여기에 작성!!!)
```

* scatter plot(산점도), boxplot 등 시각화하여 각 Attribute의 값에 대한 설명

In [ ]:
# "scatterplot3d", "plotly","PerformanceAnalytics", "boxplot" 활용


* 적절한 반응변수와 설명변수의 설정

```
반응변수 :
설명변수 :
설정한 이유 : 
```

* Train / Test set 분리 후 회귀식 추정

In [ ]:
# "caTools::sample.split", "caret:"sample.split", "lm" 활용


* 회귀분석 결과 해석(이론 강의 참고)


In [ ]:
# "summary", "plotRelWeights" 활용


* Feature selection(변수 선택)

In [ ]:
# "lm", "update", "step" 활용


* Test set을 이용한 Predict(예측)

In [ ]:
# "predict" 활용


### **1-2. 다중회귀분석 (50점)**

* 선택한 데이터의 Attribute(속성 또는 열)에 대한 설명
```
(여기에 작성!!!)
(여기에 작성!!!)
(여기에 작성!!!)
(여기에 작성!!!)
(여기에 작성!!!)
```

* scatter plot(산점도), boxplot 등 시각화하여 각 Attribute의 값에 대한 설명

In [ ]:
# "scatterplot3d", "plotly","PerformanceAnalytics", "boxplot" 활용


* 적절한 반응변수와 설명변수의 설정

```
반응변수 :
설명변수 :
설정한 이유 : 
```

* Train / Test set 분리 후 회귀식 추정

In [ ]:
# "caTools::sample.split", "caret:"sample.split", "lm" 활용


* 회귀분석 결과 해석(이론 강의 참고)


In [ ]:
# "summary", "plotRelWeights" 활용


* Feature selection(변수 선택)

In [ ]:
# "lm", "update", "step" 활용


* Test set을 이용한 Predict(예측)

In [ ]:
# "predict" 활용
